# Tour composition

How often is ridehailing used with other modes in a single tour?

In [139]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
from nhts import estReplicateSE

In [154]:
WEIGHT = 'WTPERFIN'
TAXI_MODE = 17 # Taxi/Uber/Lyft
AUTO_MODES = [3, 4, 5, 6]

In [4]:
trips17 = pd.read_csv('../data/2017/trippub.csv')
repl17 = pd.read_csv('../data/2017/perwgt.csv')
trips17 = trips17.merge(repl17.drop(['WTTRDFIN'], 1), on=['HOUSEID', 'PERSONID'], how='left', validate='m:1')

In [56]:
# Identify tours, faster
persTrips = trips17.sort_values(['HOUSEID', 'PERSONID', 'TDTRPNUM'])

In [13]:
repWeights = [f'WTPERFIN{i}' for i in range(1, 99)]

tourComposition = []

tourInfo = None
currentModes = None
previousTripWasHomeBound = False

total = len(persTrips)
for n, row in enumerate(persTrips[['TRPTRANS', 'TDTRPNUM', 'WHYTRP1S', 'HBPPOPDN', 'HOUSEID', 'PERSONID', 'WTPERFIN', *repWeights]].itertuples()):
    if tourInfo is None or row.HOUSEID != tourInfo['HOUSEID'] or row.PERSONID != tourInfo['PERSONID'] or previousTripWasHomeBound:
        if currentModes is not None:
            # save tour
            # exclude one trip tours.
            # "If you want to take a walk around the block, take a cab. We've got work to do."
            # - Morey Amsterdam, c. 1961
            tourInfo['validTour'] = all([mode > 0 for mode in currentModes]) and len(currentModes) > 1
            tourInfo['unimodalTaxiTour'] = all([mode == TAXI_MODE for mode in currentModes])
            tourInfo['unimodalAutoTour'] = all([mode in AUTO_MODES for mode in currentModes])
            tourInfo['taxiTour'] = any([mode == TAXI_MODE for mode in currentModes])
            tourInfo['autoTour'] = any([mode in AUTO_MODES for mode in currentModes])
            tourComposition.append(tourInfo)
        
        # initialize next tour
        tourInfo = dict()
        tourInfo['WTPERFIN'] = row.WTPERFIN
        tourInfo['HOUSEID'] = row.HOUSEID
        tourInfo['PERSONID'] = row.PERSONID
        tourInfo['startingTripNumber'] = row.TDTRPNUM
        tourInfo['HBPPOPDN'] = row.HBPPOPDN

        for w in repWeights:
            tourInfo[w] = getattr(row, w) # namedtuples don't support subscripting
        currentModes = []
    
    currentModes.append(row.TRPTRANS)
    previousTripWasHomeBound = row.WHYTRP1S == 1
    
    if n % 10000 == 0:
        print(f'{n} / {total}')
            
tourComposition = pd.DataFrame.from_records(tourComposition)[[
        'HOUSEID', 'PERSONID', 'startingTripNumber', 'taxiTour', 'autoTour',
    'unimodalTaxiTour', 'unimodalAutoTour', 'validTour',
 'WTPERFIN', 'WTPERFIN1', 'WTPERFIN10', 'HBPPOPDN',
       'WTPERFIN11', 'WTPERFIN12', 'WTPERFIN13', 'WTPERFIN14',
       'WTPERFIN15', 'WTPERFIN16', 'WTPERFIN17', 'WTPERFIN18',
       'WTPERFIN19', 'WTPERFIN2', 'WTPERFIN20', 'WTPERFIN21',
       'WTPERFIN22', 'WTPERFIN23', 'WTPERFIN24', 'WTPERFIN25',
       'WTPERFIN26', 'WTPERFIN27', 'WTPERFIN28', 'WTPERFIN29',
       'WTPERFIN3', 'WTPERFIN30', 'WTPERFIN31', 'WTPERFIN32',
       'WTPERFIN33', 'WTPERFIN34', 'WTPERFIN35', 'WTPERFIN36',
       'WTPERFIN37', 'WTPERFIN38', 'WTPERFIN39', 'WTPERFIN4',
       'WTPERFIN40', 'WTPERFIN41', 'WTPERFIN42', 'WTPERFIN43',
       'WTPERFIN44', 'WTPERFIN45', 'WTPERFIN46', 'WTPERFIN47',
       'WTPERFIN48', 'WTPERFIN49', 'WTPERFIN5', 'WTPERFIN50',
       'WTPERFIN51', 'WTPERFIN52', 'WTPERFIN53', 'WTPERFIN54',
       'WTPERFIN55', 'WTPERFIN56', 'WTPERFIN57', 'WTPERFIN58',
       'WTPERFIN59', 'WTPERFIN6', 'WTPERFIN60', 'WTPERFIN61',
       'WTPERFIN62', 'WTPERFIN63', 'WTPERFIN64', 'WTPERFIN65',
       'WTPERFIN66', 'WTPERFIN67', 'WTPERFIN68', 'WTPERFIN69',
       'WTPERFIN7', 'WTPERFIN70', 'WTPERFIN71', 'WTPERFIN72',
       'WTPERFIN73', 'WTPERFIN74', 'WTPERFIN75', 'WTPERFIN76',
       'WTPERFIN77', 'WTPERFIN78', 'WTPERFIN79', 'WTPERFIN8',
       'WTPERFIN80', 'WTPERFIN81', 'WTPERFIN82', 'WTPERFIN83',
       'WTPERFIN84', 'WTPERFIN85', 'WTPERFIN86', 'WTPERFIN87',
       'WTPERFIN88', 'WTPERFIN89', 'WTPERFIN9', 'WTPERFIN90',
       'WTPERFIN91', 'WTPERFIN92', 'WTPERFIN93', 'WTPERFIN94',
       'WTPERFIN95', 'WTPERFIN96', 'WTPERFIN97', 'WTPERFIN98']].copy()

0 / 923572
10000 / 923572
20000 / 923572
30000 / 923572
40000 / 923572
50000 / 923572
60000 / 923572
70000 / 923572
80000 / 923572
90000 / 923572
100000 / 923572
110000 / 923572
120000 / 923572
130000 / 923572
140000 / 923572
150000 / 923572
160000 / 923572
170000 / 923572
180000 / 923572
190000 / 923572
200000 / 923572
210000 / 923572
220000 / 923572
230000 / 923572
240000 / 923572
250000 / 923572
260000 / 923572
270000 / 923572
280000 / 923572
290000 / 923572
300000 / 923572
310000 / 923572
320000 / 923572
330000 / 923572
340000 / 923572
350000 / 923572
360000 / 923572
370000 / 923572
380000 / 923572
390000 / 923572
400000 / 923572
410000 / 923572
420000 / 923572
430000 / 923572
440000 / 923572
450000 / 923572
460000 / 923572
470000 / 923572
480000 / 923572
490000 / 923572
500000 / 923572
510000 / 923572
520000 / 923572
530000 / 923572
540000 / 923572
550000 / 923572
560000 / 923572
570000 / 923572
580000 / 923572
590000 / 923572
600000 / 923572
610000 / 923572
620000 / 923572
630000

In [162]:
taxiTours = tourComposition[tourComposition.validTour & tourComposition.taxiTour]
est = np.sum(taxiTours[~taxiTours.unimodalTaxiTour].WTPERFIN) / np.sum(taxiTours.WTPERFIN)
se = estReplicateSE(
    lambda i: np.sum(taxiTours[~taxiTours.unimodalTaxiTour][f'WTPERFIN{i}']) / np.sum(taxiTours[f'WTPERFIN{i}']),
    est,
    2017
)

print(f'{est * 100:.2f}% +/- {se * 100 * 1.96:.2f} of tours that include a taxi/ridehail are multimodal')

74.89% +/- 4.28 of tours that include a taxi/ridehail are multimodal


In [17]:
for dens in np.sort(trips17.HBPPOPDN.unique()):
    taxiTours = tourComposition[tourComposition.validTour & tourComposition.taxiTour & (tourComposition.HBPPOPDN == dens)]
    est = np.sum(taxiTours[~taxiTours.unimodalTaxiTour].WTPERFIN) / np.sum(taxiTours.WTPERFIN)
    se = estReplicateSE(
        lambda i: np.sum(taxiTours[~taxiTours.unimodalTaxiTour][f'WTPERFIN{i}']) / np.sum(taxiTours[f'WTPERFIN{i}']),
        est,
        2017
    )

    print(f'{dens}: {est * 100:.2f}% +/- {se * 100 * 1.96:.2f} of tours that include a taxi/ridehail are multimodal')

50: 54.21% +/- 45.02 of tours that include a taxi/ridehail are multimodal
300: 54.75% +/- 28.80 of tours that include a taxi/ridehail are multimodal
750: 82.21% +/- 18.44 of tours that include a taxi/ridehail are multimodal
1500: 73.23% +/- 16.77 of tours that include a taxi/ridehail are multimodal
3000: 70.36% +/- 18.14 of tours that include a taxi/ridehail are multimodal
7000: 68.58% +/- 13.09 of tours that include a taxi/ridehail are multimodal
17000: 79.41% +/- 8.05 of tours that include a taxi/ridehail are multimodal
30000: 82.64% +/- 6.25 of tours that include a taxi/ridehail are multimodal


In [163]:
autoTours = tourComposition[tourComposition.validTour & tourComposition.autoTour]
est = np.sum(autoTours[~autoTours.unimodalAutoTour].WTPERFIN) / np.sum(autoTours.WTPERFIN)
se = estReplicateSE(
    lambda i: np.sum(autoTours[~autoTours.unimodalAutoTour][f'WTPERFIN{i}']) / np.sum(autoTours[f'WTPERFIN{i}']),
    est,
    2017
)

print(f'{est * 100:.2f}% +/- {se * 100 * 1.96:.2f} of tours that include an auto are multimodal')

7.74% +/- 0.18 of tours that include an auto are multimodal
